In [22]:
#!pip install virtualenv


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
#!virtualenv preojectDPenv

created virtual environment CPython3.8.2.final.0-64 in 6446ms
  creator CPython3Windows(dest=C:\Users\Franz Schramm\source\repos\MasterThesis\preojectDPenv, clear=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=C:\Users\Franz Schramm\AppData\Local\pypa\virtualenv)
    added seed packages: pip==20.2.3, setuptools==50.3.0, wheel==0.35.1
  activators BashActivator,BatchActivator,FishActivator,PowerShellActivator,PythonActivator,XonshActivator


In [3]:
#!python -m ipykernel install --user --name=preojectDPenv
!conda list

Der Befehl "conda" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [13]:
!pip install -r requirementsDP.txt 
!pip install daal==2021.4.0
!pip install clyent==1.2.1
#!pip install opacus --user

  Using cached torch-1.13.0-cp38-cp38-win_amd64.whl (167.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
functorch 2.0.0 requires torch<2.1,>=2.0, but you have torch 1.13.0 which is incompatible.
torchtext 0.13.0 requires torch==1.12.0, but you have torch 1.13.0 which is incompatible.
torchaudio 2.0.1 requires torch==2.0.0, but you have torch 1.13.0 which is incompatible.
lightning 20220621 requires fsspec==2022.01.0, but you have fsspec 2022.8.2 which is incompatible.
lightning 20220621 requires torch<=1.11.0,>=1.9.*, but you have torch 1.13.0 which is incompatible.


In [1]:
!pip freeze > requirementsUninstall.txt

In [ ]:
!pip uninstall pytorch-lightning

In [1]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

#os.chdir("../../..")

In [ ]:
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [11]:
from pytorch_forecasting.models import TemporalFusionTransformer
import torch
from pytorch_forecasting.data.examples import get_stallion_data
from pytorch_forecasting.data import TimeSeriesDataSet
from torch.utils.data import DataLoader
from opacus import PrivacyEngine

from torchmetrics import Metric

class MSELoss(Metric):
    def __init__(self):
        super().__init__()
        self.add_state("sum_squared_error", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("n_samples", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        se = (preds - target) ** 2
        self.sum_squared_error += se.sum()
        self.n_samples += target.numel()

    def compute(self):
        mse = self.sum_squared_error.float() / self.n_samples
        return mse

# get the example data
data = get_stallion_data()

# add time index
data["time_idx"] = data["date"].dt.year * 12 + data["date"].dt.month
data["time_idx"] -= data["time_idx"].min()
# add additional features
# categories have to be strings
data["month"] = data.date.dt.month.astype(str).astype("category")
data["log_volume"] = np.log(data.volume + 1e-8)
data["avg_volume_by_sku"] = (
    data
    .groupby(["time_idx", "sku"], observed=True)
    .volume.transform("mean")
)
data["avg_volume_by_agency"] = (
    data
    .groupby(["time_idx", "agency"], observed=True)
    .volume.transform("mean")
)
# we want to encode special days as one variable and 
# thus need to first reverse one-hot encoding
special_days = [
    "easter_day", "good_friday", "new_year", "christmas",
    "labor_day", "independence_day", "revolution_day_memorial",
    "regional_games", "fifa_u_17_world_cup", "football_gold_cup",
    "beer_capital", "music_fest"
]
data[special_days] = (
    data[special_days]
    .apply(lambda x: x.map({0: "-", 1: x.name}))
    .astype("category")
)

max_prediction_length = 6
max_encoder_length = 24
training_cutoff = data["time_idx"].max() - max_prediction_length


"""
# create the time series dataset
max_encoder_length = 24
max_prediction_length = 4
training_cutoff = "2014-09-01"
validation_cutoff = "2014-11-01"
training = TimeSeriesDataSet(
    data[lambda x: x.date <= training_cutoff],
    time_idx="timeseries",
    target="volume",
    categorical_encoders={"event_name": None},
    group_ids=["agency", "sku"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["agency", "sku"],
    static_reals=["year", "month"],
    time_varying_known_categoricals=["event_name"],
    time_varying_known_reals=["temperature"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=["volume"],
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)
"""

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="volume",
    group_ids=["agency", "sku"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["agency", "sku"],
    static_reals=["avg_population_2017", "avg_yearly_household_income_2017"],
    time_varying_known_categoricals=["special_days", "month"],
    variable_groups={"special_days": special_days},  # group of categorical variables can be treated as one variable
    time_varying_known_reals=["time_idx", "price_regular", "discount_in_percent"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "volume",
        "log_volume",
        "industry_volume",
        "soda_volume",
        "avg_max_temp",
        "avg_volume_by_agency",
        "avg_volume_by_sku",
    ],
    target_normalizer=GroupNormalizer(
        groups=["agency", "sku"], transformation="softplus"
    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)


validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

batch_size = 128
train_dataloader = DataLoader(training, batch_size=batch_size, shuffle=True, drop_last=True)
val_dataloader = DataLoader(validation, batch_size=batch_size, shuffle=False, drop_last=True)


mse_loss = MSELoss()
# create the TemporalFusionTransformer model
model = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,
    loss=mse_loss,
)

# create the privacy engine
epsilon = 1.0
delta = 1e-5
max_grad_norm = 0.1
noise_multiplier=0.1
optimizer = torch.optim.Adam(model.parameters(), lr=0.5)
privacy_engine = PrivacyEngine()
model, optimizer, dataloader = privacy_engine.make_private(
    module = model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    noise_multiplier=noise_multiplier,
    max_grad_norm=max_grad_norm
    )

privacy_engine.attach(optimizer=model.optim)

# train the model
num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_dataloader:
        x, y = batch
        # apply differential privacy to the gradients
        optimizer.zero_grad()
        loss = model(x, y)
        loss.backward()
        optimizer.step()
        privacy_engine.step()

    # evaluate the model on the validation set
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_dataloader:
            x, y = batch
            y_pred = model.predict(x)
            val_loss += model.loss(y_pred, y).item()
    val_loss /= len(val_dataloader)

    print(f"Epoch {epoch+1}/{num_epochs}, validation loss: {val_loss:.4f}")

OSError: [WinError 127] Die angegebene Prozedur wurde nicht gefunden

In [10]:
#!pip install torch torchvision torchaudio opacus --no-cache-dir
!pip install torch==1.13.0 torchvision


  Using cached torch-1.13.0-cp38-cp38-win_amd64.whl (167.3 MB)
     ---------------------------------------- 1.2/1.2 MB 3.3 MB/s eta 0:00:00
  Using cached torchvision-0.14.1-cp38-cp38-win_amd64.whl (1.1 MB)
  Using cached torchvision-0.14.0-cp38-cp38-win_amd64.whl (1.1 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.9.0
    Uninstalling torchvision-0.9.0:
      Successfully uninstalled torchvision-0.9.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
functorch 2.0.0 requires torch<2.1,>=2.0, but you have torch 1.13.0 which is incompatible.
torchtext 0.13.0 requires torch==1.12.0, but you have torch 1.13.0 which is incompatible.
torchaudio 2.0.1 requires torch==2.0.0, but you have torch 1.13.0 which is incompatible.
lightning 20220621 requires fsspec==2022.01.0, but you have fsspec 2022.8.2 which is incompatible.
lightning 20220621 requires torch<=1.11.0,>=1.9.*, but you have torch 1.13.0 which is incompatible.


In [45]:
!pip install opacus

     -------------------------------------- 167.3/167.3 MB 3.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0
    Uninstalling torch-1.12.0:
      Successfully uninstalled torch-1.12.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
functorch 2.0.0 requires torch<2.1,>=2.0, but you have torch 1.13.0 which is incompatible.
torchvision 0.15.1 requires torch==2.0.0, but you have torch 1.13.0 which is incompatible.
torchtext 0.13.0 requires torch==1.12.0, but you have torch 1.13.0 which is incompatible.
torchaudio 2.0.1 requires torch==2.0.0, but you have torch 1.13.0 which is incompatible.
lightning 20220621 requires fsspec==2022.01.0, but you have fsspec 2022.8.2 which is incompatible.
lightning 20220621 requires torch<=1.11.0,>=1.9.*, but you have torch 1.13.0 which is incompatible.


In [16]:
data.head()

,agency,sku,volume,date,industry_volume,soda_volume,avg_max_temp,price_regular,price_actual,discount,...,labor_day,independence_day,revolution_day_memorial,regional_games,fifa_u_17_world_cup,football_gold_cup,beer_capital,music_fest,discount_in_percent,timeseries
0,Agency_22,SKU_01,52.2720,2013-01-01,492612703,718394219,25.845238,1168.903668,1069.166193,99.737475,...,0,0,0,0,0,0,0,0,8.532566,0
238,Agency_37,SKU_04,0.0000,2013-01-01,492612703,718394219,26.505000,1852.273642,1611.466298,240.807344,...,0,0,0,0,0,0,0,0,13.000635,5
237,Agency_59,SKU_03,812.9214,2013-01-01,492612703,718394219,22.219737,1270.795012,1197.184260,73.610752,...,0,0,0,0,0,0,0,0,5.792496,9
236,Agency_11,SKU_01,316.4400,2013-01-01,492612703,718394219,25.360000,1176.155397,1082.757488,93.397909,...,0,0,0,0,0,0,0,0,7.940950,14
235,Agency_05,SKU_05,420.9093,2013-01-01,492612703,718394219,24.079012,1327.003396,1207.822992,119.180404,...,0,0,0,0,0,0,0,0,8.981168,22
